# Demo of Snowpark end-to-end Machine Learning


The dataset is from https://archive-beta.ics.uci.edu/dataset/222/bank+marketing


**Run 00_Load_demo_data.ipynb to upload the Parquet files used for this Notebook**

In [36]:
# Imports 
import snowflake.snowpark as S
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark import Window

import joblib
import cachetools
import io
import os

import json

# Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

import sqlparse

# Print the version of Snowpark we are using
print(f"Using Snowpark: {S.__version__}")

Using Snowpark: 1.2.0


Helper functions for nicer printing of Snowparkd dataframe schema, SQL and to generate a correlation matrix.

In [37]:
# Helper functions for nicer printing
def print_sql(df):
    for query in df.queries['queries']:
        print(sqlparse.format(query, reindent=True))

def print_schema(df):
    print("schema:")
    for col in df.schema.fields:
        print(f" |-- {col.name}: {col.datatype} (Nullable: {col.nullable})")

def shape(df):
    return (df.count(), len(df.columns))

Connect to Snowflake

This example is using a JSON file with the following structure
```
{
    "account":"MY SNOWFLAKE ACCOUNT",
    "user": "MY USER",
    "password":"MY PASSWORD",
    "role":"MY ROLE",
    "warehouse":"MY WH",
    "database":"MY DB",
    "schema":"MY SCHEMA"
}

```

In [38]:
with open('../creds.json') as f:
    connection_parameters = json.load(f)

session = Session.builder.configs(connection_parameters).create()
print("Current role: " + session.get_current_role() + ", Current schema: " + session.get_fully_qualified_current_schema() + ", Current WH: " + session.get_current_warehouse())

Current role: "SYSADMIN", Current schema: "DEMO_DB"."PUBLIC", Current WH: "COMPUTE_WH"


In [39]:
# Parameters
source_path = "@SOURCE_FILES/BANK_MARKETING" # Where the source parquet files are stored
sp_udf_stage = "BANK_STAGE" # Name of the stage to used for storing the code for the SP and UDF , as well the trained model files

## Loading of source data
### Loading Parquet files with inferring the schema.

Start to check that the source files is on the stage

In [41]:
session.sql(f"ls {source_path}").select('"name"').show(30, max_width=150)

-------------------------------------------------------
|"name"                                               |
-------------------------------------------------------
|source_files/BANK_MARKETING/data_apr.snappy.parquet  |
|source_files/BANK_MARKETING/data_aug.snappy.parquet  |
|source_files/BANK_MARKETING/data_dec.snappy.parquet  |
|source_files/BANK_MARKETING/data_feb.snappy.parquet  |
|source_files/BANK_MARKETING/data_jan.snappy.parquet  |
|source_files/BANK_MARKETING/data_jul.snappy.parquet  |
|source_files/BANK_MARKETING/data_jun.snappy.parquet  |
|source_files/BANK_MARKETING/data_mar.snappy.parquet  |
|source_files/BANK_MARKETING/data_may.snappy.parquet  |
|source_files/BANK_MARKETING/data_nov.snappy.parquet  |
|source_files/BANK_MARKETING/data_oct.snappy.parquet  |
|source_files/BANK_MARKETING/data_sep.snappy.parquet  |
-------------------------------------------------------



Take a peak in the files

In [42]:
session.sql("create or replace temp file format parq1 type='PARQUET'").collect()
session.sql(f"select $1 from {source_path} (file_format=>parq1 )").show(2)

-------------------------------
|"$1"                         |
-------------------------------
|{                            |
|  "AGE": 46,                 |
|  "BALANCE": -90,            |
|  "CAMPAIGN": 1,             |
|  "CONTACT": "telephone",    |
|  "DAY": 17,                 |
|  "DEFAULT": "no",           |
|  "DURATION": 270,           |
|  "EDUCATION": "secondary",  |
|  "HOUSING": "no",           |
|  "JOB": "technician",       |
|  "LOAN": "yes",             |
|  "MARITAL": "divorced",     |
|  "MONTH": "oct",            |
|  "OUTCOME": "unknown",      |
|  "PDAYS": -1,               |
|  "PREVIOUS": 0,             |
|  "Y": "yes"                 |
|}                            |
|{                            |
|  "AGE": 30,                 |
|  "BALANCE": 1785,           |
|  "CAMPAIGN": 1,             |
|  "CONTACT": "unknown",      |
|  "DAY": 17,                 |
|  "DEFAULT": "no",           |
|  "DURATION": 235,           |
|  "EDUCATION": "tertiary",   |
|  "HOUS

Loading Parquet files with inferring the schema.

In [43]:
df_reader = session.read.parquet(source_path)
df_reader.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"AGE"  |"JOB"         |"MARITAL"  |"EDUCATION"  |"DEFAULT"  |"BALANCE"  |"HOUSING"  |"LOAN"  |"CONTACT"  |"DAY"  |"MONTH"  |"DURATION"  |"CAMPAIGN"  |"PDAYS"  |"PREVIOUS"  |"OUTCOME"  |"Y"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|23     |student       |single     |secondary    |no         |922        |no         |no      |cellular   |1      |sep      |81          |5           |99       |3           |other      |no   |
|32     |entrepreneur  |single     |tertiary     |no         |59649      |no         |no      |cellular   |1      |sep      |69          |2           |-1       |0           |unknown    |no   |
|43     |management    |divorced   

The df_reader datafarme is reading the files from stage when used, we can check this by looking at the SQL it generates

In [44]:
print_sql(df_reader)

CREATE SCOPED
TEMPORARY FILE FORMAT If NOT EXISTS "DEMO_DB"."PUBLIC".SNOWPARK_TEMP_FILE_FORMAT_KZNHO5F7JH TYPE = PARQUET
SELECT $1:"AGE"::NUMBER(38, 0) AS "AGE",
          $1:"JOB"::TEXT AS "JOB",
             $1:"MARITAL"::TEXT AS "MARITAL",
                $1:"EDUCATION"::TEXT AS "EDUCATION",
                   $1:"DEFAULT"::TEXT AS "DEFAULT",
                      $1:"BALANCE"::NUMBER(38, 0) AS "BALANCE",
                         $1:"HOUSING"::TEXT AS "HOUSING",
                            $1:"LOAN"::TEXT AS "LOAN",
                               $1:"CONTACT"::TEXT AS "CONTACT",
                                  $1:"DAY"::NUMBER(38, 0) AS "DAY",
                                     $1:"MONTH"::TEXT AS "MONTH",
                                        $1:"DURATION"::NUMBER(38, 0) AS "DURATION",
                                           $1:"CAMPAIGN"::NUMBER(38, 0) AS "CAMPAIGN",
                                              $1:"PDAYS"::NUMBER(38, 0) AS "PDAYS",
                      

To load the data into a Snowflake table we can use copy_into_table.

It will create the table if it not exists, using the infered schema, and if the table exists it will append the data. However, Snowflake keeps track of what files it has loaded so it does not load the same file twice, by dropping the table we ensure that the files are loaded

In [45]:
session.sql("DROP TABLE IF EXISTS bank_marketing_v2").collect()
df_reader.copy_into_table("bank_marketing_v2")

[Row(file='source_files/BANK_MARKETING/data_sep.snappy.parquet', status='LOADED', rows_parsed=579, rows_loaded=579, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None),
 Row(file='source_files/BANK_MARKETING/data_dec.snappy.parquet', status='LOADED', rows_parsed=214, rows_loaded=214, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None),
 Row(file='source_files/BANK_MARKETING/data_oct.snappy.parquet', status='LOADED', rows_parsed=738, rows_loaded=738, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None),
 Row(file='source_files/BANK_MARKETING/data_jan.snappy.parquet', status='LOADED', rows_parsed=1403, rows_loaded=1403, error_limit=1, errors_seen=0, first_error=None, first_error_line=None, first_error_character=None, first_error_column_name=None),
 Row(file='sou

Create a Snowpark Dataframe using the new table

In [46]:
df_bank_marketing = session.table("bank_marketing_v2")
display(f"Dataframe shape: {shape(df_bank_marketing)}")
df_bank_marketing.show()

'Dataframe shape: (45211, 17)'

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"AGE"  |"JOB"         |"MARITAL"  |"EDUCATION"  |"DEFAULT"  |"BALANCE"  |"HOUSING"  |"LOAN"  |"CONTACT"  |"DAY"  |"MONTH"  |"DURATION"  |"CAMPAIGN"  |"PDAYS"  |"PREVIOUS"  |"OUTCOME"  |"Y"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|34     |technician    |single     |secondary    |no         |1866       |no         |no      |cellular   |4      |nov      |95          |1           |-1       |0           |unknown    |no   |
|41     |services      |married    |secondary    |no         |523        |yes        |yes     |telephone  |4      |nov      |5           |1           |-1       |0           |unknown    |no   |
|60     |management    |married    

In [47]:
print_sql(df_bank_marketing)

SELECT *
FROM (bank_marketing_v2)


### Data understanding

Start with verifying datatypes, simple put we will treat charcter columns as categorical

In [48]:
print_schema(df_bank_marketing)

schema:
 |-- AGE: LongType() (Nullable: True)
 |-- JOB: StringType() (Nullable: True)
 |-- MARITAL: StringType() (Nullable: True)
 |-- EDUCATION: StringType() (Nullable: True)
 |-- DEFAULT: StringType() (Nullable: True)
 |-- BALANCE: LongType() (Nullable: True)
 |-- HOUSING: StringType() (Nullable: True)
 |-- LOAN: StringType() (Nullable: True)
 |-- CONTACT: StringType() (Nullable: True)
 |-- DAY: LongType() (Nullable: True)
 |-- MONTH: StringType() (Nullable: True)
 |-- DURATION: LongType() (Nullable: True)
 |-- CAMPAIGN: LongType() (Nullable: True)
 |-- PDAYS: LongType() (Nullable: True)
 |-- PREVIOUS: LongType() (Nullable: True)
 |-- OUTCOME: StringType() (Nullable: True)
 |-- Y: StringType() (Nullable: True)


DAY is stored as a number but can be threaded as categorical, fixed number of days in months, and by changing the data type to character we will do that.

In [49]:
df_bank_marketing_prep = df_bank_marketing.with_column("DAY", F.to_varchar(F.col("DAY")))
print_schema(df_bank_marketing_prep)

schema:
 |-- AGE: LongType() (Nullable: True)
 |-- JOB: StringType() (Nullable: True)
 |-- MARITAL: StringType() (Nullable: True)
 |-- EDUCATION: StringType() (Nullable: True)
 |-- DEFAULT: StringType() (Nullable: True)
 |-- BALANCE: LongType() (Nullable: True)
 |-- HOUSING: StringType() (Nullable: True)
 |-- LOAN: StringType() (Nullable: True)
 |-- CONTACT: StringType() (Nullable: True)
 |-- MONTH: StringType() (Nullable: True)
 |-- DURATION: LongType() (Nullable: True)
 |-- CAMPAIGN: LongType() (Nullable: True)
 |-- PDAYS: LongType() (Nullable: True)
 |-- PREVIOUS: LongType() (Nullable: True)
 |-- OUTCOME: StringType() (Nullable: True)
 |-- Y: StringType() (Nullable: True)
 |-- DAY: StringType() (Nullable: True)


Get basic statistics about the categorical and numeric columns

In [50]:
df_bank_marketing_prep.describe().show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"AGE"               |"JOB"    |"MARITAL"  |"EDUCATION"  |"DEFAULT"  |"BALANCE"           |"HOUSING"  |"LOAN"  |"CONTACT"  |"MONTH"  |"DURATION"          |"CAMPAIGN"          |"PDAYS"             |"PREVIOUS"         |"OUTCOME"  |"Y"    |"DAY"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |45211.0             |45211    |45211      |45211        |45211      |45211.0             |45211      |45211   |45211      |45211    |45211.0             |45211.0             |45211.0             |45211.0          

Create variables with our categorical, numeric and target columns names so we can use them with encoders and scalers

In [51]:
cat_cols = [c.name for c in df_bank_marketing_prep.schema.fields if (type(c.datatype) == T.StringType) & (c.name != 'Y')]
numeric_types = [T.DecimalType, T.LongType, T.DoubleType, T.FloatType, T.IntegerType]
num_cols = [c.name for c in df_bank_marketing_prep.schema.fields if type(c.datatype) in numeric_types]
target_col = "Y"

Distribution of target values

In [52]:
df_bank_marketing_prep.group_by(target_col).count().show()

-----------------
|"Y"  |"COUNT"  |
-----------------
|yes  |5289     |
|no   |39922    |
-----------------



Frequency tables for each categorical feature

In [53]:
for col in cat_cols:
    display(df_bank_marketing_prep.select(F.count_distinct(col).as_(f"{col} distinct values")).show())
    display(df_bank_marketing_prep.group_by(col).count()\
                                .select(col, (F.call_function("RATIO_TO_REPORT", F.col("COUNT")).over() * 100).as_("% observations") )\
                                .sort(F.col("% observations").desc()).show(31))

-------------------------
|"JOB distinct values"  |
-------------------------
|12                     |
-------------------------



None

------------------------------------
|"JOB"          |"% observations"  |
------------------------------------
|blue-collar    |21.525700         |
|management     |20.919700         |
|technician     |16.803400         |
|admin.         |11.437500         |
|services       |9.188000          |
|retired        |5.007600          |
|self-employed  |3.492500          |
|entrepreneur   |3.289000          |
|unemployed     |2.882000          |
|housemaid      |2.742700          |
|student        |2.074700          |
|unknown        |0.637000          |
------------------------------------



None

-----------------------------
|"MARITAL distinct values"  |
-----------------------------
|3                          |
-----------------------------



...

None

-------------------------
|"DAY distinct values"  |
-------------------------
|31                     |
-------------------------



None

----------------------------
|"DAY"  |"% observations"  |
----------------------------
|20     |6.087000          |
|18     |5.105000          |
|21     |4.481200          |
|17     |4.288800          |
|6      |4.273300          |
|5      |4.224600          |
|14     |4.087500          |
|8      |4.074200          |
|28     |4.047700          |
|7      |4.018900          |
|19     |3.886200          |
|29     |3.859700          |
|15     |3.766800          |
|12     |3.545600          |
|13     |3.505800          |
|30     |3.463800          |
|9      |3.452700          |
|11     |3.271300          |
|4      |3.196100          |
|16     |3.129800          |
|2      |2.859900          |
|27     |2.479500          |
|3      |2.386600          |
|26     |2.289300          |
|23     |2.076900          |
|22     |2.001700          |
|25     |1.858000          |
|31     |1.422200          |
|10     |1.159000          |
|24     |0.988700          |
|1      |0.712200          |
--------------

None

Relationship between each of the categorical features and the target column

In [54]:
for col in cat_cols:
    window = Window.partition_by(col)
    display(df_bank_marketing_prep.group_by(col, F.col(target_col))\
                                .count()\
                                .select(col, F.col(target_col), (F.call_function("RATIO_TO_REPORT", F.col("COUNT")).over(window) * 100).as_("percentage"))\
                                .pivot(target_col, ['no', 'yes']).agg(F.sum("percentage")).show(50))


-----------------------------------------
|"JOB"          |"'no'"     |"'yes'"    |
-----------------------------------------
|services       |91.117000  |8.883000   |
|blue-collar    |92.725000  |7.275000   |
|technician     |88.943000  |11.057000  |
|student        |71.322000  |28.678000  |
|unknown        |88.194400  |11.805600  |
|retired        |77.208500  |22.791500  |
|management     |86.244400  |13.755600  |
|housemaid      |91.209700  |8.790300   |
|unemployed     |84.497300  |15.502700  |
|admin.         |87.797300  |12.202700  |
|self-employed  |88.157100  |11.842900  |
|entrepreneur   |91.728300  |8.271700   |
-----------------------------------------



None

-------------------------------------
|"MARITAL"  |"'no'"     |"'yes'"    |
-------------------------------------
|divorced   |88.054500  |11.945500  |
|single     |85.050800  |14.949200  |
|married    |89.876500  |10.123500  |
-------------------------------------



None

---------------------------------------
|"EDUCATION"  |"'no'"     |"'yes'"    |
---------------------------------------
|secondary    |89.440600  |10.559400  |
|primary      |91.373500  |8.626500   |
|unknown      |86.429700  |13.570300  |
|tertiary     |84.993600  |15.006400  |
---------------------------------------



...

None

-------------------------------------
|"OUTCOME"  |"'no'"     |"'yes'"    |
-------------------------------------
|other      |83.315200  |16.684800  |
|success    |35.274700  |64.725300  |
|unknown    |90.838500  |9.161500   |
|failure    |87.390300  |12.609700  |
-------------------------------------



None

---------------------------------
|"DAY"  |"'no'"     |"'yes'"    |
---------------------------------
|8      |89.087900  |10.912100  |
|2      |85.924200  |14.075800  |
|15     |86.024700  |13.975300  |
|29     |92.607400  |7.392600   |
|11     |87.762000  |12.238000  |
|18     |90.121300  |9.878700   |
|19     |93.056300  |6.943700   |
|17     |90.923200  |9.076800   |
|6      |90.631500  |9.368500   |
|23     |86.581500  |13.418500  |
|12     |84.778500  |15.221500  |
|30     |82.694800  |17.305200  |
|16     |86.431100  |13.568900  |
|7      |91.359400  |8.640600   |
|13     |84.795000  |15.205000  |
|20     |93.023300  |6.976700   |
|24     |86.129800  |13.870200  |
|4      |84.083000  |15.917000  |
|1      |72.049700  |27.950300  |
|27     |86.619100  |13.380900  |
|28     |92.185800  |7.814200   |
|9      |88.533000  |11.467000  |
|26     |88.792300  |11.207700  |
|22     |82.983400  |17.016600  |
|25     |84.166700  |15.833300  |
|3      |83.503200  |16.496800  |
|5      |88.74

None

### Model training

In [55]:
def train_response_model(X, y, cat_cols, num_cols):
    
    # One Hot Encoder transformer for categorical columns
    cat_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    # Standard scaler for numerical columns
    num_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    # Combine into a column transformer
    preprocessor = ColumnTransformer(
      [
            ('num', num_transformer, num_cols),
            ('cat', cat_transformer, cat_cols),
        ],  verbose_feature_names_out=False,
    )
    
    # Create a pipeline with the column transformer and training of a Random Forrest Classifier
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(n_jobs=-1))])
    rfc_model = pipe.fit(X, y)
    return rfc_model

In [56]:
pd_train = df_bank_marketing.sample(frac=0.10).to_pandas()

X = pd_train[[*cat_cols, *num_cols]]
y = pd_train["Y"]
model = train_response_model(X, y, cat_cols, num_cols)

In [57]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['AGE', 'BALANCE', 'DURATION',
                                                   'CAMPAIGN', 'PDAYS',
                                                   'PREVIOUS']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['JOB', 'MARITAL',
                                                   'EDUCATION', 'DEFAULT',
                                                   'HOUSING', 'LOAN', 'CONTACT',
                                                   'MONTH', 'OUTCOME',
                                                   'DAY'])],
                                   verbose_feature_names_out=False)),
                ('classifier', RandomForestClassifier(n_jobs=-1))])

Test the model

In [58]:
def save_file_to_stage(snf_session, obj, file_name, stage_path):
    import io
    import joblib
    
    file_path = stage_path + file_name
    
    input_stream = io.BytesIO()
    joblib.dump(obj, input_stream)
    snf_session._conn._cursor.upload_stream(input_stream, file_path)
    
    return file_path

def train_bank(snf_session: Session, stage: str) -> dict:
    from datetime import datetime
    
    df_train, df_test = snf_session.table("bank_marketing_v2").random_split([0.8, 0.2])
    
    # Get the categorical, numerical and target column
    cat_cols = [c.name for c in df_train.schema.fields if (type(c.datatype) == T.StringType) & (c.name != 'Y')]
    numeric_types = [T.DecimalType, T.LongType, T.DoubleType, T.FloatType, T.IntegerType]
    num_cols = [c.name for c in df_train.schema.fields if type(c.datatype) in numeric_types]
    target_col = "Y"
    X_cols = [*cat_cols, *num_cols]
    
    pd_train = df_train.to_pandas()
    X = pd_train[X_cols]
    y = pd_train[target_col]
    
    # fit the pipeline
    model = train_response_model(X, y, cat_cols, num_cols)
    
    # Test the model
    pd_test = df_test.to_pandas()
    
    X_test = pd_test[X_cols]
    y_test = pd_test[target_col]

    y_pred = model.predict(X_test)
    
    # Create a dict with some test scores based on test data to return
    ret_dict = {"f1_score" : f1_score(y_test, y_pred, average='macro')
                  , "precision_score": precision_score(y_test, y_pred, average='macro')
                  , "recall_score": recall_score(y_test, y_pred, average='macro')
                  , "accuracy_score" : accuracy_score(y_test, y_pred)}

    now = datetime.now()
    # Save the model to stage
    save_path = now.strftime("%Y-%m-%d-%H%M%S")
    model_path = save_file_to_stage(snf_session, model, 'rfc_bank_model.joblib', f'@{stage}/{save_path}/')
    ret_dict['model_path'] = model_path
    return ret_dict

Create a stage for storing the code for the SP, UDF and model file.

If you do not want to remove the old models, skip this step

In [59]:
session.sql(f"CREATE OR REPLACE STAGE {sp_udf_stage}").collect()

[Row(status='Stage area BANK_STAGE successfully created.')]

Create the Store Procedure in Snowflake.
The **sproc** function returns a callable object that can be used to call the stored procedure.

In [60]:
session.clear_imports()
session.clear_packages()
session.add_packages('snowflake-snowpark-python','pandas', 'scikit-learn==1.1.3', 'joblib')
train_bank_sp = F.sproc(func=train_bank,name="train_bank", is_permanent = True, replace= True, stage_location = f'{sp_udf_stage}/bank/sp/', session=session)

The version of package snowflake-snowpark-python in the local environment is 1.2.0, which does not fit the criteria for the requirement snowflake-snowpark-python. Your UDF might not work when the package version is different between the server and your local environment


Run the training Stored Procedure in Snowflake

In [61]:
sp_dict = json.loads(train_bank_sp(session, f'{sp_udf_stage}/bank/model'))
sp_dict

{'accuracy_score': 0.9031511679153812,
 'f1_score': 0.7161033663929085,
 'model_path': '@BANK_STAGE/bank/model/2023-03-20-025600/rfc_bank_model.joblib',
 'precision_score': 0.7895891097330634,
 'recall_score': 0.6789827094992672}

Check that the model is saved to the Snowflake stage

In [62]:
session.sql(f"ls @{sp_udf_stage}/bank/model").show(max_width=150)

-----------------------------------------------------------------------------------------------------------------------------------------------
|"name"                                                         |"size"    |"md5"                             |"last_modified"                |
-----------------------------------------------------------------------------------------------------------------------------------------------
|bank_stage/bank/model/2023-03-20-025600/rfc_bank_model.joblib  |52034992  |b4d980d50ac559c8c16a2a29990d33a4  |Mon, 20 Mar 2023 09:56:02 GMT  |
-----------------------------------------------------------------------------------------------------------------------------------------------



Deploy model as a UDF

In [63]:
def deploy_model(snf_session, udf_name, udf_stage, model_name, import_model_path, features):
    
    # Function to load the model file, using cachetools makes sure file is only loaded once
    @cachetools.cached(cache={})
    def read_file(filename):
        import joblib
        import sys
        import os

        import_dir = sys._xoptions.get("snowflake_import_directory")
        if import_dir:
            with open(os.path.join(import_dir, filename), 'rb') as file:
                m = joblib.load(file)
                return m
    
    # Use a vectorized udf, gets maximum 100 rows at the time
    @F.udf(name = udf_name, max_batch_size=100, is_permanent = True, stage_location = udf_stage, imports = [import_model_path]
           , packages = ['pandas', 'scikit-learn==1.1.3', 'cachetools'], replace = True, session = snf_session)
    def survived(pd_input: T.PandasDataFrame[str, str, str, str, str, str, str, str, str, str, int, int, int, int, int, int]) -> T.PandasSeries[str]:
        # Make sure we have the columns in the expected order in the Pandas Dataframe
        model = read_file(model_name)
        pd_input.columns = features
        prediction = model.predict(pd_input)
        return prediction


Call the deployment function

In [64]:
features = [*cat_cols, *num_cols]
deploy_model(session, "predict_response", f"@{sp_udf_stage}/bank/udf/", "rfc_bank_model.joblib" ,sp_dict['model_path'], features)

Test the UDF with our data

In [65]:
# Generate a list of columns to use with the UDF
input_cols = [F.col(col) for col in features]

df_response_scores = df_bank_marketing.select(*input_cols, F.col(target_col), F.call_function("predict_response", *input_cols).alias('PREDICTION'))
df_response_scores.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"JOB"         |"MARITAL"  |"EDUCATION"  |"DEFAULT"  |"HOUSING"  |"LOAN"  |"CONTACT"  |"MONTH"  |"OUTCOME"  |"DAY"  |"AGE"  |"BALANCE"  |"DURATION"  |"CAMPAIGN"  |"PDAYS"  |"PREVIOUS"  |"Y"  |"PREDICTION"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|technician    |single     |secondary    |no         |no         |no      |cellular   |nov      |unknown    |4      |34     |1866       |95          |1           |-1       |0           |no   |no            |
|services      |married    |secondary    |no         |yes        |yes     |telephone  |nov      |unknown    |4      |41     |523        |5           |1           |-1   

Using the crosstab method allows us to do a quick confusion matrix

In [66]:
df_response_scores.crosstab("Y", "PREDICTION").show()

--------------------------
|"Y"  |"'yes'"  |"'no'"  |
--------------------------
|yes  |4624     |665     |
|no   |216      |39706   |
--------------------------



In [67]:
session.close()